### Kaggle : Titanic - Machine Learning from Disaster

# Import libraries

In [61]:
# Data Manimulation
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Model Building
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score

# Data Visualisation
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

# Import data

In [62]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [63]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [64]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data examination

In [65]:
train.shape

(891, 12)

In [66]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [67]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [69]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# Missing values clean-up

### Drop Cabin column

In [70]:
train = train.drop('Cabin', axis = 1)
test = test.drop('Cabin', axis = 1)
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S


### Fill 2 missing values in Embarked column with most common value

In [71]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [72]:
train = train.fillna({"Embarked": "S"})

In [73]:
train.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

### Drop rows with missing age

In [74]:
train = train.dropna()

In [75]:
train.shape

(714, 11)

### Insert column mean value for missing values in test dataset

In [76]:
test = test.fillna({"Fare" : train.Fare.mean()})
test = test.fillna({"Age" : train.Age.mean()})

### Final number of missing values

In [77]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [78]:
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

# Categorical values clean-up

In [79]:
label_encoder = OrdinalEncoder()

train['Embarked_cat'] = label_encoder.fit_transform(train[['Embarked']])
test['Embarked_cat'] = label_encoder.fit_transform(test[['Embarked']])

train[['Embarked', 'Embarked_cat']].head(20)

,Embarked,Embarked_cat
0,S,2.0
1,C,0.0
2,S,2.0
3,S,2.0
4,S,2.0
6,S,2.0
7,S,2.0
8,S,2.0
9,C,0.0
10,S,2.0


In [80]:
train['Sex_cat'] = label_encoder.fit_transform(train[['Sex']])
test['Sex_cat'] = label_encoder.fit_transform(test[['Sex']])

train[['Sex', 'Sex_cat']].head(5)

,Sex,Sex_cat
0,male,1.0
1,female,0.0
2,female,0.0
3,female,0.0
4,male,1.0


### Drop old categorical columns

In [81]:
train = train.drop('Sex', axis = 1)
test = test.drop('Sex', axis = 1)

train = train.drop('Embarked', axis = 1)
test = test.drop('Embarked', axis = 1)

train.head(3)

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Embarked_cat,Sex_cat
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,2.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,2.0,0.0


### Drop columns with irrelevant categorical values

In [82]:
train = train.drop('Name', axis = 1)
test = test.drop('Name', axis = 1)

train = train.drop('Ticket', axis = 1)
test = test.drop('Ticket', axis = 1)

train.head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_cat,Sex_cat
0,1,0,3,22.0,1,0,7.2500,2.0,1.0
1,2,1,1,38.0,1,0,71.2833,0.0,0.0
2,3,1,3,26.0,0,0,7.9250,2.0,0.0


# Building model

In [83]:
y_train = train['Survived']
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [84]:
X_train = train.drop('PassengerId', axis=1)
X_train = X_train.drop('Survived', axis=1)

X_test = test.drop('PassengerId', axis=1)

X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked_cat,Sex_cat
0,3,22.0,1,0,7.2500,2.0,1.0
1,1,38.0,1,0,71.2833,0.0,0.0
2,3,26.0,0,0,7.9250,2.0,0.0
3,1,35.0,1,0,53.1000,2.0,0.0
4,3,35.0,0,0,8.0500,2.0,1.0


In [85]:
X_train.shape, y_train.shape, X_test.shape

((714, 7), (714,), (418, 7))

## Decision Tree

In [86]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
Y_pred_tree = model.predict(X_test)
acc_decision_tree = round(model.score(X_train, y_train) * 100, 2)
acc_decision_tree

98.6

## Random Forest

In [87]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_pred_forest = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

98.6

# Cross - validation

In [88]:
cv_scores_decision_tree = cross_val_score(model, X_train, y_train, 
                            cv=5,
                            scoring='accuracy')

cv_scores_random_forest = cross_val_score(random_forest, X_train, y_train, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy for decision tree: " ,cv_scores_decision_tree.mean(), " , random forest: "  ,cv_scores_random_forest.mean())

Cross-validation accuracy for decision tree:  0.7647493351718705  , random forest:  0.7970058110903182


## Output file

In [89]:
Y_pred_forest.shape , test.PassengerId.shape

((418,), (418,))

In [90]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': Y_pred_forest})
output.to_csv('output/output.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
